# Importing Liberaries

In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# import nltk
# nltk.download('stopwords')  # download if not already done

import re
from nltk.corpus import stopwords

# Data Pre-Processing

In [3]:
df = pd.read_csv("Sentiment.csv")
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
newdf = df[["candidate","text","sentiment"]]
newdf

,candidate,text,sentiment
0,No candidate mentioned,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,Scott Walker,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,No candidate mentioned,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,No candidate mentioned,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,Donald Trump,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...,...
13866,No candidate mentioned,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,Mike Huckabee,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,Ted Cruz,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,Donald Trump,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [5]:
def remove_punctuation(text):
  """Removes punctuation from the given text."""
  return re.sub(r'[^\w\s]', '', text)

def remove_stopwords(text):
  """Removes stop words from the given text."""
  stop_words = set(stopwords.words('english'))  # Download stopwords if needed
  return ' '.join([word for word in text.split() if word not in stop_words])

def preprocess_text(text):
  """Preprocesses text by removing punctuation, converting to lowercase, and removing stop words."""
  text = text.lower()  # Convert to lowercase
  text = remove_punctuation(text)
  text = remove_stopwords(text)
  return text

def vectorized_preprocess(text_list):
    return [preprocess_text(text) for text in text_list]

df['text_cleaned'] = np.vectorize(preprocess_text)(df['text'].to_numpy())


In [6]:
newdf = df[["candidate","text_cleaned","sentiment"]]
newdf

,candidate,text_cleaned,sentiment
0,No candidate mentioned,rt nancyleegrahn everyone feel climate change ...,Neutral
1,Scott Walker,rt scottwalker didnt catch full gopdebate last...,Positive
2,No candidate mentioned,rt tjmshow mention tamir rice gopdebate held c...,Neutral
3,No candidate mentioned,rt robgeorge carly fiorina trending hours deba...,Positive
4,Donald Trump,rt danscavino gopdebate w realdonaldtrump deli...,Positive
...,...,...,...
13866,No candidate mentioned,rt cappy_yarbrough love see men never faced pr...,Negative
13867,Mike Huckabee,rt georgehenryw thought huckabee exceeded expe...,Positive
13868,Ted Cruz,rt lrihendry tedcruz president always tell tru...,Positive
13869,Donald Trump,rt jrehling gopdebate donald trump says doesnt...,Negative


In [7]:
text_length = newdf['text_cleaned'].apply(lambda x: len(x.split()))
text_length.quantile([0.25, 0.50, 0.75, 0.90, 1])


0.25     9.0
0.50    11.0
0.75    13.0
0.90    15.0
1.00    25.0
Name: text_cleaned, dtype: float64

In [8]:
# Combine all text from the text_cleaned column into a single string
all_text = ' '.join(newdf['text_cleaned'])

# Split the combined text into individual words
words = all_text.split()

# Convert the list of words into a set to get unique words
unique_words = set(words)

# Get the total number of unique words
num_unique_words = len(unique_words)

num_unique_words

19293

In [9]:
# Maximum sequence length (number of words)
max_len = 25

# 90 percentile sequence length (number of words)
len = 15

# Number of most frequent words to consider (vocabulary size)
vocab_size = num_unique_words

In [10]:
# Create Tokenizer
tokenizer = Tokenizer(num_words=vocab_size)

# Fit Tokenizer on Text Data
tokenizer.fit_on_texts(newdf['text_cleaned'].tolist())

# Convert Text to Sequences
sequences = tokenizer.texts_to_sequences(newdf['text_cleaned'].tolist())

# Add padding to Sequences
padded_sequences = pad_sequences(sequences, maxlen=len, padding='post')

# One hot encoding for target
le = LabelEncoder()
labels = le.fit_transform(newdf['sentiment'])
labels = to_categorical(labels, num_classes=3)

# Model Building

In [11]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim=len))
model.add(SpatialDropout1D(0.2))  # Helps prevent overfitting
model.add(LSTM(64, return_sequences=True))  # Process sequences
model.add(LSTM(32))  # Extract features
model.add(Dense(3, activation='softmax'))  # Output layer for 3 sentiment classes

# Compile model
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])

# Train-validation split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True)

# Model training
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test), callbacks=[early_stopping])


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 11096 samples, validate on 2775 samples
Epoch 1/40
11096/11096 [==============================] - 14s 1ms/sample - loss: 0.8696 - acc: 0.6260 - val_loss: 0.7790 - val_acc: 0.6537
Epoch 2/40
11096/11096 [==============================] - 13s 1ms/sample - loss: 0.6932 - acc: 0.6975 - val_loss: 0.7581 - val_acc: 0.6739
Epoch 3/40
11096/11096 [==============================] - 13s 1ms/sample - loss: 0.5792 - acc: 0.7638 - val_loss: 0.7640 - val_acc: 0.6732
Epoch 4/40
11096/11096 [==============================] - 13s 1ms/sample - loss: 0.4706 - acc: 0.8143 - val_loss: 0.8230 - val_acc: 0.6620
Epoch 5/40
11096/11096 [==============================] - 13s 1ms/sample - loss: 0.3764 - acc: 0.8624 

In [12]:
# Evaluate model on validation set
loss, accuracy = model.evaluate(X_test, y_test)
print('Final Validation Accuracy:', round(accuracy,4), 'and Final Validation Loss:', round(loss,4))

2775/2775 [==============================] - 1s 223us/sample - loss: 0.7581 - acc: 0.6739
Final Validation Accuracy: 0.6739 and Final Validation Loss: 0.7581


In [13]:
# Saving the model
model.save('Twitter Sentiment Predictor.h5')